<<9日K值<20買，>80賣出>>

設定要關注的股票stock
抓取庫存資料之股票代碼a
抓取此檔股票今天的K值
if 庫存裡沒有此檔股票: 
    if K<20:  
        提醒使用者現在K值小於20，建議買進  
elif 庫存裡有此檔股票(stock==a):
    抓取持有成本  
    if 現價-持有價格>停利標準:  
        提醒使用者現在已經達到預設的停利標準，建議脫手  
    elif 現價-持有價格>停損標準:  
        提醒使用者現在已經達到預設的停損標準，建議脫手  
    elif K<20:  
        提醒使用者現在K值小於20，建議買進  
    elif K>80:  
        提醒使用者現在K值大於80，建議賣出  

In [1]:
from bs4 import BeautifulSoup
# basic
import numpy as np
import pandas as pd
import pymongo
# visual
import matplotlib.pyplot as plt
%matplotlib inline
#time
import datetime 

#talib
import talib
from talib import abstract

#gui

import requests
import re
import io

import schedule
import time

In [2]:
#Installation
#!pip install line-bot-sdk
import tkinter as tk

In [3]:
#Linebot連接
from __future__ import print_function
import time
from linebot import (LineBotApi, WebhookHandler, exceptions)
from linebot.exceptions import (InvalidSignatureError)
from linebot.models import *
import schedule
from pymongo import MongoClient
import urllib.parse
import datetime
import requests
from bs4 import BeautifulSoup

line_bot_api = LineBotApi('b2ZZ5S9d8Uvtl6CqAQiSS2ykRn4fBehUEnj0i1PitOEe2bN+yPW7XbnZyZaYj6asN/pinquk9P2+7evxROlazavRnRUmS1NVqmvW9OVCFOTM28Pz71e7bKjbTlHTMS60pR/6TrkDjEgs1kHXLSF7HQdB04t89/1O/w1cDnyilFU=')
yourid='Uc52a42025be949ba1b1e0654d6536122'

In [4]:
#爬取股票
def crawl_price(stock_id):
    now = int(datetime.datetime.now().timestamp())+86400
    url = "https://query1.finance.yahoo.com/v7/finance/download/" + stock_id + "?period1=0&period2=" + str(now) + "&interval=1d&events=history&crumb=hP2rOschxO0"

    response = requests.post(url)
    
    f = io.StringIO(response.text)
    df = pd.read_csv(f, index_col='Date', parse_dates=['Date'] )
    return df

In [5]:
#抓取虛擬交易資料
def get_webservice(url):
    list_req = requests.get(url)
    soup = BeautifulSoup(list_req.content,'html.parser')
    r = soup.find('string').text
    return r

In [6]:
#爬取 K值
def getK(stock):
    
    df = crawl_price(stock)
    start_time = datetime.datetime.now()-datetime.timedelta(days=365)
    end_time = datetime.date.today()
    close = df.Close[start_time:end_time].dropna().astype(float)
    close2 = round(close, 2)
    openp = df.Open[start_time:end_time]
    high = df.High[start_time:end_time]
    low = df.Low[start_time:end_time]
    volumn = df.Volume[start_time:end_time]
    stock = {
        'close':close.dropna().astype(float),
        'open':openp.dropna().astype(float),
        'high':high.dropna().astype(float),
        'low':low.dropna().astype(float),
        'volume': volumn.dropna().astype(float),
    }
    def talib2df(talib_output):
        if type(talib_output) == list:
            ret = pd.DataFrame(talib_output).transpose()
        else:
            ret = pd.Series(talib_output)
        ret.index = stock['close'].index
        return ret;

    data = talib2df(talib.abstract.STOCH(stock,fastk_period=9,
                    slowk_period=3,
                    slowk_matype=0,
                    slowd_period=3,
                    slowd_matype=0))

    data.reset_index(inplace=True)
    date = data.iloc[0:,0]
    K = data.iloc[0:,1]
    a=[]
    for i in range(len(K)):
        a.append(K[i])
    return a[-1]

In [7]:
#爬取現在的股價
def getPriceNow(stock):
    
    url2 = "http://61.220.30.176/WebOrder/GVETransacs.asmx/QueryQuote5Price?compcode=" + stock
    client2 = get_webservice(url2)
    try:
        import xml.etree.cElementTree as ET
    except ImportError:
        import xml.etree.ElementTree as ET
    r2 = ET.fromstring(client2)
    stock_now = str(r2[0][0].text)#抓取現價
    return stock_now

In [8]:
#提醒
def job(stock,sProfit,sLoss):
    stockl = stock.lower() #將.TW轉成.tw才能爬取虛擬交易所的庫存
    stock_now = getPriceNow(stock)
    stock_now = float(stock_now)
    a = getK(stock)
    sProfit = float(sProfit)
    sLoss = float(sLoss)
    url = "http://61.220.30.176/WebOrder/GVEAccount.asmx/QueryTodayPositionGve3XML_NS?TokenString=143986D99078C7FA6A0B5BCD8C00ACA4A1DB04385D50EFF4682053C7A0AA4979D7B81BE534ECF4C969E3EA65DFF55137F00EEF9BA3C9FF02C78BA63C37C6202EA694BE201140613AF586FADA1560C84FD5517892C838E79199922D9DDF92DE7626D7BE97ADF465278B4ABD03F7CDF9573B578E7BA64604142854EF2CF90DE997F75D73B2D6499FB20F6841F13751C5C906CB71B300D30C76&Language=TC&SubTotalItem=&SortItem=AssetCode%20Asc"
    client = get_webservice(url)
    try:
        import xml.etree.cElementTree as ET
    except ImportError:
        import xml.etree.ElementTree as ET
    r = ET.fromstring(client)
    stock_list=[]
    price=[]
    for country in r.findall('PortfolioAsset'):
            AssetCode = country.get('AssetCode')
            if AssetCode == stockl: #判斷使用者要找的股票是否有在庫存裡
                stock_list.append(AssetCode) #將找到的股票代碼存進stock_list
                Cost = country.get('Cost')
                price.append(Cost) #將找到的股票成本價格存進stock_list
                break #找到就跳出迴圈，stock_list裡面的最後一筆資料是使用者要找的
    if len(stock_list): #判斷stock_list裡面有沒有股票
        teststock = stock_list[-1] #股票代碼
        ownprice = float(price[-1]) #股票的成本價
        comprice = stock_now - ownprice #現價-成本價
        if stockl == teststock:
            if stock_now >= ownprice + sProfit:
                get = "持有股票已經達到預設的停利標準，建議脫手"
                print(get)
                line_bot_api.push_message(yourid, TextSendMessage(text=get))
            elif stock_now <= ownprice - sLoss:
                get = "持有股票已經達到預設的停損標準，建議脫手"
                print(get)
                line_bot_api.push_message(yourid, TextSendMessage(text=get))
            elif a<20:
                get = "現在K值小於20，建議買進"
                print(get)
                line_bot_api.push_message(yourid, TextSendMessage(text=get))
            elif a>80:
                get = "現在K值大於80，建議賣出"
                print(get)
                line_bot_api.push_message(yourid, TextSendMessage(text=get))
            else:
                get = "現在K值於20~80之間，不建議做交易"
                print(get)
                line_bot_api.push_message(yourid, TextSendMessage(text=get))
    else:
        if a<20:
            get = "K值: "+str(a)+"\n" + "現在K值小於20，建議買進"
            print(get)
            line_bot_api.push_message(yourid, TextSendMessage(text=get))                
            
        else:
            get = "K值: "+str(a)+"\n" + "現在K值大於20，不建議做交易"
            print(get)
            line_bot_api.push_message(yourid, TextSendMessage(text=get))

In [9]:
from tkinter import *
running = True
def start(stockid,sProfit,sLoss):
    second_5_j = schedule.every(5).seconds.do(job,stockid,sProfit,sLoss)
    def scanning():
        if running:  
            schedule.run_pending()
            time.sleep(1)
        root.after(100, scanning)

    root = Tk()
    root.title("Title")
    root.geometry("200x200")

    app = Frame(root)
    app.grid()
    
    text1 = Label(root,text='自動提醒中....')
    text1.grid()
    root.after(100, scanning)
    root.mainloop()